In [23]:
import random
import csv

# ---- Config ----
categories = ["X", "Y", "Z"]
num_dmps_per_category = 10
num_participants = 10  # total desired
batch_size = num_dmps_per_category  # 10 participants per batch
random.seed()  # set random.seed(42) for reproducibility

assignments = []
participant_id = 1

# Generate exactly num_participants rows
while participant_id <= num_participants:
    # Create one shuffled base permutation for this batch
    base = list(range(1, num_dmps_per_category + 1))
    random.shuffle(base)

    # Shifts ensure different numbers per row
    shifts = {"X": 0, "Y": 1, "Z": 2}

    # Fill this batch (or until we reach total participants)
    for i in range(batch_size):
        if participant_id > num_participants:
            break  # stop if we already reached the total
        row = {}
        for cat in categories:
            num = base[(i + shifts[cat]) % num_dmps_per_category]
            row[cat] = f"{cat}_{num}.md"

        assignments.append({
            "Participant number": participant_id,
            "DMP1": row["X"],
            "DMP2": row["Y"],
            "DMP3": row["Z"],
        })
        participant_id += 1

# ---- Save to CSV ----
with open("participant_dmp_assignments.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["Participant number", "DMP1", "DMP2", "DMP3"])
    writer.writeheader()
    writer.writerows(assignments)

print("✅ Saved to participant_dmp_assignments.csv with", len(assignments), "records")

# ---- Validation ----
def check_row_number_uniqueness(assignments):
    ok = True
    for row in assignments:
        nums = []
        for k in ("DMP1","DMP2","DMP3"):
            _, num = row[k].split("_")
            nums.append(int(num.replace(".md","")))
        if len(set(nums)) != 3:
            ok = False
            print(f"❌ Row {row['Participant number']}: duplicate number(s) across categories -> {nums}")
    if ok:
        print("✅ All rows have distinct numbers across categories.")
    return ok

check_row_number_uniqueness(assignments)


✅ Saved to participant_dmp_assignments.csv with 10 records
✅ All rows have distinct numbers across categories.


True

In [27]:
import os
import random
import csv
from math import ceil

# ---------------- Config ----------------
CATEGORIES = ["X", "Y", "Z"]
NUM_DMPS_PER_CATEGORY = 10
NUM_PARTICIPANTS = 30
BATCH_SIZE = NUM_DMPS_PER_CATEGORY  # 10 participants per batch
LINKS_CSV = "dmp_links.csv"         # master link table you edit by hand
ASSIGN_CSV = "participant_dmp_assignments.csv"
ASSIGN_WITH_LINKS_CSV = "participant_dmp_assignments_with_links.csv"

random.seed()  # set e.g. random.seed(42) for reproducibility

# ---------------- Assignment generation (rotation ensures distinct numbers per row) ----------------
assignments = []
participant_id = 1
num_batches = ceil(NUM_PARTICIPANTS / BATCH_SIZE)

for b in range(num_batches):
    remaining = NUM_PARTICIPANTS - participant_id + 1
    this_batch_size = min(BATCH_SIZE, remaining)

    base = list(range(1, NUM_DMPS_PER_CATEGORY + 1))
    random.shuffle(base)

    shifts = {"X": 0, "Y": 1, "Z": 2}  # fixed rotation

    for i in range(this_batch_size):
        row = {}
        for cat in CATEGORIES:
            num = base[(i + shifts[cat]) % NUM_DMPS_PER_CATEGORY]
            row[cat] = f"{cat}_{num}.md"

        assignments.append({
            "Participant number": participant_id,
            "DMP1": row["X"],
            "DMP2": row["Y"],
            "DMP3": row["Z"],
        })
        participant_id += 1

# Save base assignments
with open(ASSIGN_CSV, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["Participant number", "DMP1", "DMP2", "DMP3"])
    writer.writeheader()
    writer.writerows(assignments)
print(f"✅ Saved {ASSIGN_CSV} ({len(assignments)} records)")

# ---------------- Master link table maintenance ----------------
def load_links(path):
    """Return dict: { 'X_1.md': 'https://...' } if file exists, else {}."""
    if not os.path.exists(path):
        return {}
    links = {}
    with open(path, newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row.get("dmp_md", "").strip()
            link = row.get("pdf_link", "").strip()
            if name:
                links[name] = link
    return links

def save_links(mapping, path):
    """Write mapping back (stable sorted by filename)."""
    with open(path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["dmp_md", "pdf_link"])
        writer.writeheader()
        for name in sorted(mapping.keys()):
            writer.writerow({"dmp_md": name, "pdf_link": mapping[name]})

def update_link_table(dmp_names, path=LINKS_CSV):
    """
    Ensure all DMP names exist in the master CSV.
    Preserve existing links; add new rows with empty link.
    """
    links = load_links(path)
    added = 0
    for name in dmp_names:
        if name not in links:
            links[name] = ""   # you fill this manually later
            added += 1
    if added or not os.path.exists(path):
        save_links(links, path)
    return links, added

# Gather all unique DMP filenames used in assignments
all_dmps = set()
for row in assignments:
    all_dmps.add(row["DMP1"])
    all_dmps.add(row["DMP2"])
    all_dmps.add(row["DMP3"])

# Update master link CSV (preserves your manual edits)
links_map, num_added = update_link_table(all_dmps, LINKS_CSV)
print(f"✅ Updated {LINKS_CSV}: {len(links_map)} total rows ({num_added} new).")
print("   👉 Open this CSV and paste your Google Drive PDF links in the 'pdf_link' column.")

# ---------------- Join links into a convenience CSV ----------------
with open(ASSIGN_WITH_LINKS_CSV, "w", newline="") as f:
    fieldnames = ["Participant number",
                  "DMP1", "DMP1_link",
                  "DMP2", "DMP2_link",
                  "DMP3", "DMP3_link"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in assignments:
        writer.writerow({
            "Participant number": row["Participant number"],
            "DMP1": row["DMP1"],
            "DMP1_link": links_map.get(row["DMP1"], ""),
            "DMP2": row["DMP2"],
            "DMP2_link": links_map.get(row["DMP2"], ""),
            "DMP3": row["DMP3"],
            "DMP3_link": links_map.get(row["DMP3"], ""),
        })
print(f"✅ Saved {ASSIGN_WITH_LINKS_CSV} (joined with links)")

✅ Saved participant_dmp_assignments.csv (30 records)
✅ Updated dmp_links.csv: 30 total rows (0 new).
   👉 Open this CSV and paste your Google Drive PDF links in the 'pdf_link' column.
✅ Saved participant_dmp_assignments_with_links.csv (joined with links)


In [28]:
import csv
import re
from collections import defaultdict

# ---- Config (keep in sync with your generator) ----
CATEGORIES = ["X", "Y", "Z"]
NUM_DMPS_PER_CATEGORY = 10
BATCH_SIZE = NUM_DMPS_PER_CATEGORY           # expecting batches of 10
CSV_PATH = "participant_dmp_assignments_with_links.csv" # change if needed

# ---- Helpers ----
file_pattern = re.compile(r"^([A-Za-z]+)_(\d+)\.md$")  # e.g., X_7.md

def parse_cell(cell):
    """Return (category, number) if valid, else raise ValueError."""
    m = file_pattern.match(cell.strip())
    if not m:
        raise ValueError(f"Bad filename format: {cell!r}")
    cat, num = m.group(1), int(m.group(2))
    if cat not in CATEGORIES:
        raise ValueError(f"Unknown category {cat!r} in {cell!r}")
    if not (1 <= num <= NUM_DMPS_PER_CATEGORY):
        raise ValueError(f"Number out of range in {cell!r} (must be 1..{NUM_DMPS_PER_CATEGORY})")
    return cat, num

def validate_row(row):
    """
    Validate a single assignment row.
    - Has DMP1..DMP3
    - Each DMP matches pattern
    - Exactly one from each category (no duplicate categories)
    """
    errors = []
    dmps = []
    for key in ("DMP1","DMP2","DMP3"):
        if key not in row or not row[key]:
            errors.append(f"Missing {key}")
            continue
        try:
            dmps.append(parse_cell(row[key]))
        except ValueError as e:
            errors.append(str(e))

    # If parsing failed for any, stop further checks
    if errors:
        return False, errors, dmps

    cats = [c for c,_ in dmps]
    if len(set(cats)) != 3:
        errors.append(f"Duplicate/missing categories in row: {cats}. Expected one each of {CATEGORIES}.")
    if set(cats) != set(CATEGORIES):
        missing = set(CATEGORIES) - set(cats)
        extra = set(cats) - set(CATEGORIES)
        if missing:
            errors.append(f"Missing categories: {sorted(missing)}")
        if extra:
            errors.append(f"Unexpected categories: {sorted(extra)}")

    return (len(errors) == 0), errors, dmps

def validate_batches(rows):
    """
    Check that in each 10-participant batch, each category covers numbers 1..10 exactly once.
    """
    ok = True
    for start in range(0, len(rows), BATCH_SIZE):
        batch = rows[start:start+BATCH_SIZE]
        # Track seen numbers per category within the batch
        seen = {cat: set() for cat in CATEGORIES}
        for r in batch:
            for key in ("DMP1","DMP2","DMP3"):
                try:
                    cat, num = parse_cell(r[key])
                    if cat in seen:
                        seen[cat].add(num)
                except Exception:
                    # Row-level errors will be reported elsewhere
                    pass

        expected = set(range(1, NUM_DMPS_PER_CATEGORY + 1))
        batch_idx = start // BATCH_SIZE + 1
        for cat in CATEGORIES:
            if set(seen[cat]) != expected:
                ok = False
                missing = sorted(expected - seen[cat])
                extra   = sorted(seen[cat] - expected)
                msg = []
                if missing: msg.append(f"missing {missing}")
                if extra:   msg.append(f"extra {extra}")
                print(f"❌ Batch {batch_idx} category {cat}: does not cover 1..{NUM_DMPS_PER_CATEGORY} exactly once ({'; '.join(msg)})")
        if all(set(seen[c]) == expected for c in CATEGORIES):
            print(f"✅ Batch {batch_idx}: perfect coverage for all categories.")
    return ok

def overall_distribution(rows):
    """
    Optional: show how many times each DMP number appears per category across ALL participants.
    (With 30 participants in 3 equal batches, each number should appear exactly 3 times per category.)
    """
    counts = {cat: defaultdict(int) for cat in CATEGORIES}
    for r in rows:
        for key in ("DMP1","DMP2","DMP3"):
            try:
                cat, num = parse_cell(r[key])
                counts[cat][num] += 1
            except Exception:
                pass
    print("\n— Overall distribution per category —")
    for cat in CATEGORIES:
        print(f"{cat}: ", {n: counts[cat][n] for n in range(1, NUM_DMPS_PER_CATEGORY+1)})

# ---- Load CSV ----
with open(CSV_PATH, newline="") as f:
    reader = csv.DictReader(f)
    rows = list(reader)

# ---- Row-by-row validation ----
all_rows_ok = True
for r in rows:
    pid = r.get("Participant number", "?")
    ok, errors, dmps = validate_row(r)
    if ok:
        print(f"✅ Row {pid}: OK  -> {r['DMP1']}, {r['DMP2']}, {r['DMP3']}")
    else:
        all_rows_ok = False
        print(f"❌ Row {pid}:")
        for e in errors:
            print("   -", e)

# ---- Batch-level coverage validation ----
print("\n=== Batch coverage check ===")
batches_ok = validate_batches(rows)

# ---- Optional overall distribution summary ----
overall_distribution(rows)

# ---- Final verdict ----
print("\n=== Summary ===")
if all_rows_ok and batches_ok:
    print("✅ All rows valid AND all batches have perfect per-category coverage.")
elif not all_rows_ok and not batches_ok:
    print("❌ Some rows are invalid AND batch coverage is not satisfied. See messages above.")
elif not all_rows_ok:
    print("❌ Some rows are invalid. Batch coverage may still be OK; see messages above.")
else:
    print("❌ All rows valid, but batch coverage FAILED. See messages above.")


✅ Row 1: OK  -> X_2.md, Y_10.md, Z_4.md
✅ Row 2: OK  -> X_10.md, Y_4.md, Z_7.md
✅ Row 3: OK  -> X_4.md, Y_7.md, Z_8.md
✅ Row 4: OK  -> X_7.md, Y_8.md, Z_6.md
✅ Row 5: OK  -> X_8.md, Y_6.md, Z_5.md
✅ Row 6: OK  -> X_6.md, Y_5.md, Z_9.md
✅ Row 7: OK  -> X_5.md, Y_9.md, Z_1.md
✅ Row 8: OK  -> X_9.md, Y_1.md, Z_3.md
✅ Row 9: OK  -> X_1.md, Y_3.md, Z_2.md
✅ Row 10: OK  -> X_3.md, Y_2.md, Z_10.md
✅ Row 11: OK  -> X_4.md, Y_1.md, Z_3.md
✅ Row 12: OK  -> X_1.md, Y_3.md, Z_5.md
✅ Row 13: OK  -> X_3.md, Y_5.md, Z_8.md
✅ Row 14: OK  -> X_5.md, Y_8.md, Z_6.md
✅ Row 15: OK  -> X_8.md, Y_6.md, Z_9.md
✅ Row 16: OK  -> X_6.md, Y_9.md, Z_10.md
✅ Row 17: OK  -> X_9.md, Y_10.md, Z_7.md
✅ Row 18: OK  -> X_10.md, Y_7.md, Z_2.md
✅ Row 19: OK  -> X_7.md, Y_2.md, Z_4.md
✅ Row 20: OK  -> X_2.md, Y_4.md, Z_1.md
✅ Row 21: OK  -> X_1.md, Y_2.md, Z_4.md
✅ Row 22: OK  -> X_2.md, Y_4.md, Z_6.md
✅ Row 23: OK  -> X_4.md, Y_6.md, Z_3.md
✅ Row 24: OK  -> X_6.md, Y_3.md, Z_5.md
✅ Row 25: OK  -> X_3.md, Y_5.md, Z_9.md
✅ R

In [25]:
import os
import random
import csv
from math import ceil

# ---------------- Config ----------------
CATEGORIES = ["X", "Y", "Z"]
NUM_DMPS_PER_CATEGORY = 10
NUM_PARTICIPANTS = 30
BATCH_SIZE = NUM_DMPS_PER_CATEGORY  # 10 participants per batch
LINKS_CSV = "dmp_links.csv"         # master link table you edit by hand
ASSIGN_CSV = "participant_dmp_assignments.csv"
ASSIGN_WITH_LINKS_CSV = "participant_dmp_assignments_with_links.csv"

random.seed()  # set e.g. random.seed(42) for reproducibility

# ---------------- Assignment generation (rotation ensures distinct numbers per row) ----------------
assignments = []
participant_id = 1
num_batches = ceil(NUM_PARTICIPANTS / BATCH_SIZE)

for b in range(num_batches):
    remaining = NUM_PARTICIPANTS - participant_id + 1
    this_batch_size = min(BATCH_SIZE, remaining)

    base = list(range(1, NUM_DMPS_PER_CATEGORY + 1))
    random.shuffle(base)

    shifts = {"X": 0, "Y": 1, "Z": 2}  # fixed rotation

    for i in range(this_batch_size):
        row = {}
        for cat in CATEGORIES:
            num = base[(i + shifts[cat]) % NUM_DMPS_PER_CATEGORY]
            row[cat] = f"{cat}_{num}.md"

        assignments.append({
            "Participant number": participant_id,
            "DMP1": row["X"],
            "DMP2": row["Y"],
            "DMP3": row["Z"],
        })
        participant_id += 1

# Save base assignments
with open(ASSIGN_CSV, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["Participant number", "DMP1", "DMP2", "DMP3"])
    writer.writeheader()
    writer.writerows(assignments)
print(f"✅ Saved {ASSIGN_CSV} ({len(assignments)} records)")

# ---------------- Master link table maintenance ----------------
def load_links(path):
    """Return dict: { 'X_1.md': 'https://...' } if file exists, else {}."""
    if not os.path.exists(path):
        return {}
    links = {}
    with open(path, newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row.get("dmp_md", "").strip()
            link = row.get("pdf_link", "").strip()
            if name:
                links[name] = link
    return links

def save_links(mapping, path):
    """Write mapping back (stable sorted by filename)."""
    with open(path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["dmp_md", "pdf_link"])
        writer.writeheader()
        for name in sorted(mapping.keys()):
            writer.writerow({"dmp_md": name, "pdf_link": mapping[name]})

def update_link_table(dmp_names, path=LINKS_CSV):
    """
    Ensure all DMP names exist in the master CSV.
    Preserve existing links; add new rows with empty link.
    """
    links = load_links(path)
    added = 0
    for name in dmp_names:
        if name not in links:
            links[name] = ""   # you fill this manually later
            added += 1
    if added or not os.path.exists(path):
        save_links(links, path)
    return links, added

# Gather all unique DMP filenames used in assignments
all_dmps = set()
for row in assignments:
    all_dmps.add(row["DMP1"])
    all_dmps.add(row["DMP2"])
    all_dmps.add(row["DMP3"])

# Update master link CSV (preserves your manual edits)
links_map, num_added = update_link_table(all_dmps, LINKS_CSV)
print(f"✅ Updated {LINKS_CSV}: {len(links_map)} total rows ({num_added} new).")
print("   👉 Open this CSV and paste your Google Drive PDF links in the 'pdf_link' column.")

# ---------------- Join links into a convenience CSV ----------------
with open(ASSIGN_WITH_LINKS_CSV, "w", newline="") as f:
    fieldnames = ["Participant number",
                  "DMP1", "DMP1_link",
                  "DMP2", "DMP2_link",
                  "DMP3", "DMP3_link"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in assignments:
        writer.writerow({
            "Participant number": row["Participant number"],
            "DMP1": row["DMP1"],
            "DMP1_link": links_map.get(row["DMP1"], ""),
            "DMP2": row["DMP2"],
            "DMP2_link": links_map.get(row["DMP2"], ""),
            "DMP3": row["DMP3"],
            "DMP3_link": links_map.get(row["DMP3"], ""),
        })
print(f"✅ Saved {ASSIGN_WITH_LINKS_CSV} (joined with links)")


✅ Saved participant_dmp_assignments.csv (30 records)
✅ Updated dmp_links.csv: 30 total rows (30 new).
   👉 Open this CSV and paste your Google Drive PDF links in the 'pdf_link' column.
✅ Saved participant_dmp_assignments_with_links.csv (joined with links)


In [1]:
import pandas as pd

# Original table
data = {
    "Participant number": [
        "2170", "4830", "1379", "2205", "3091", "4126", "5340", "6078", "7124",
        "8053", "8296", "9012", "9185", "9427", "9963"
    ],
    "DMP1": [
        "Human_4", "Llama_1", "Human_7", "Gpt_3", "Gpt_1", "Human_6", "Human_8",
        "Human_3", "Gpt_2", "Gpt_7", "Llama_7", "Llama_8", "Gpt_9", "Llama_5", "Llama_4"
    ],
    "DMP2": [
        "Llama_10", "Human_2", "Gpt_6", "Human_9", "Human_1", "Gpt_5", "Llama_4",
        "Llama_7", "Llama_2", "Llama_8", "Gpt_7", "Gpt_1", "Human_9", "Gpt_3", "Gpt_4"
    ],
    "DMP3": [
        "Gpt_10", "Gpt_9", "Llama_6", "Llama_5", "Llama_3", "Llama_9", "Gpt_4",
        "Gpt_8", "Human_5", "Human_10", "Human_3", "Human_5", "Llama_6", "Human_2", "Human_1"
    ]
}
df = pd.DataFrame(data)

# Create user-facing anonymized table and mapping table
user_df = df.copy()
mapping_list = []

for idx, row in df.iterrows():
    pid = row['Participant number']
    for i, dmp_col in enumerate(['DMP1', 'DMP2', 'DMP3'], start=1):
        anon_code = f"{pid}_{i}"
        user_df.at[idx, dmp_col] = anon_code
        mapping_list.append({'encrypt_Code': anon_code, 'Participant number': pid, 'DMP': dmp_col, 'Actual_Model': row[dmp_col]})

mapping_df = pd.DataFrame(mapping_list)

# Save as CSV files
user_df.to_csv('participant_dmp_encrypt.csv', index=False)
mapping_df.to_csv('participant_dmp_mapping.csv', index=False)

print("Saved as participant_dmp_anonymized.csv and participant_dmp_mapping.csv")


Saved as participant_dmp_anonymized.csv and participant_dmp_mapping.csv


In [3]:
import os
import shutil
import pandas as pd

# Paths
mapping_csv = 'participant_dmp_mapping.csv'
input_folder = 'Clean DMP'       # Update to your folder with original files
output_folder = 'Clean DMP-encrypt'     # Update to your desired output folder

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load mapping
mapping_df = pd.read_csv(mapping_csv)

for _, row in mapping_df.iterrows():
    actual_model = row['Actual_Model']
    encrypt_code = row['encrypt_Code']
    
    for ext in ['.docx', '.md']:
        src_file = os.path.join(input_folder, actual_model + ext)
        dst_file = os.path.join(output_folder, encrypt_code + ext)
        
        if os.path.exists(src_file):
            shutil.copy2(src_file, dst_file)
            print(f"Copied: {src_file} -> {dst_file}")
        else:
            print(f"WARNING: File does not exist: {src_file}")

print("All files processed.")


Copied: Clean DMP\Human_4.docx -> Clean DMP-encrypt\2170_1.docx
Copied: Clean DMP\Human_4.md -> Clean DMP-encrypt\2170_1.md
Copied: Clean DMP\Llama_10.docx -> Clean DMP-encrypt\2170_2.docx
Copied: Clean DMP\Llama_10.md -> Clean DMP-encrypt\2170_2.md
Copied: Clean DMP\Gpt_10.docx -> Clean DMP-encrypt\2170_3.docx
Copied: Clean DMP\Gpt_10.md -> Clean DMP-encrypt\2170_3.md
Copied: Clean DMP\Llama_1.docx -> Clean DMP-encrypt\4830_1.docx
Copied: Clean DMP\Llama_1.md -> Clean DMP-encrypt\4830_1.md
Copied: Clean DMP\Human_2.docx -> Clean DMP-encrypt\4830_2.docx
Copied: Clean DMP\Human_2.md -> Clean DMP-encrypt\4830_2.md
Copied: Clean DMP\Gpt_9.docx -> Clean DMP-encrypt\4830_3.docx
Copied: Clean DMP\Gpt_9.md -> Clean DMP-encrypt\4830_3.md
Copied: Clean DMP\Human_7.docx -> Clean DMP-encrypt\1379_1.docx
Copied: Clean DMP\Human_7.md -> Clean DMP-encrypt\1379_1.md
Copied: Clean DMP\Gpt_6.docx -> Clean DMP-encrypt\1379_2.docx
Copied: Clean DMP\Gpt_6.md -> Clean DMP-encrypt\1379_2.md
Copied: Clean DM

In [2]:
import os
import shutil
import pandas as pd

# Optional: Install docx2pdf if not already installed
! pip install docx2pdf

from docx2pdf import convert

# Paths
mapping_csv = 'participant_dmp_mapping.csv'
input_folder = 'Clean DMP'
output_folder = 'Clean DMP-encrypt'

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load mapping
mapping_df = pd.read_csv(mapping_csv)

# Copy and rename files
for _, row in mapping_df.iterrows():
    actual_model = row['Actual_Model']
    encrypt_code = row['encrypt_Code']
    
    for ext in ['.docx', '.md']:
        src_file = os.path.join(input_folder, actual_model + ext)
        dst_file = os.path.join(output_folder, encrypt_code + ext)
        
        if os.path.exists(src_file):
            shutil.copy2(src_file, dst_file)
            print(f"Copied: {src_file} -> {dst_file}")
        else:
            print(f"WARNING: File does not exist: {src_file}")

print("All files processed.")

# --- Convert .docx files in output folder to .pdf ---
for fname in os.listdir(output_folder):
    if fname.endswith('.docx'):
        docx_path = os.path.join(output_folder, fname)
        pdf_path = os.path.splitext(docx_path)[0] + '.pdf'
        try:
            convert(docx_path, pdf_path)
            print(f"Converted: {docx_path} -> {pdf_path}")
        except Exception as e:
            print(f"ERROR converting {docx_path}: {e}")

print("All .docx files converted to PDF.")


ModuleNotFoundError: No module named 'docx2pdf'